<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_02_2_multi_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 2: Code Generation**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 2 Material

* Part 2.1: Prompting for Code Generation [[Video]](https://www.youtube.com/watch?v=HVId6kYKKgQ) [[Notebook]](t81_559_class_02_1_dev.ipynb)
* **Part 2.2: Handling Revision Prompts** [[Video]](https://www.youtube.com/watch?v=APpV46tplXA) [[Notebook]](t81_559_class_02_2_multi_prompt.ipynb)
* Part 2.3: Using a LLM to Help Debug [[Video]](https://www.youtube.com/watch?v=VPqSNb38QK0) [[Notebook]](t81_559_class_02_3_llm_debug.ipynb)
* Part 2.4: Tracking Prompts in Software Development [[Video]](https://www.youtube.com/watch?v=oUFUuYfvXZU) [[Notebook]](t81_559_class_02_4_software_eng.ipynb)
* Part 2.5: Limits of LLM Code Generation [[Video]](https://www.youtube.com/watch?v=dKtRI0LZSyY) [[Notebook]](t81_559_class_02_5_code_gen_limits.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

Note: using Google CoLab
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 812.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
  A

# 2.2: Handling Revision Prompts

Previously, we just sent one prompt to the LLM, which generated code. It is possible to perform this code more conversationally. In this module, we will see how to converse with the LLM to request changes to outputted code and even help the LLM to produce a more accurate model.

We will also see that it might be beneficial to recreate your conversation as one single prompt that generates the final result. Keeping track of one prompt, rather than a conversation, that created your final code is more maintainable.

## Conversational Code Generation

We will introduce a more advanced code generation function that allows you to start the conversation to generate code and follow up with additional prompts if needed.

In future modules, we will see how to create chatbots similar to this one. We will use the code I provided to generate your code for now. This generator uses a system prompt that requests that the generated code conform to the following:

* Imports should be sorted
* Code should conform to PEP-8 formatting
* Do not mix uncompilable notes with code
* Add comments

In [2]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

MODEL = 'gpt-4o-mini'
TEMPLATE = """The following is a friendly conversation between a human and an
AI to generate Python code. If you have notes about the code, place them before
the code. Any nots about execution should follow the code. If you do mix any
notes with the code, make them comments. Add proper comments to the code.
Sort imports and follow PEP-8 formatting.

Current conversation:
{history}
Human: {input}
Code Assistant:"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

def start_conversation():
    # Initialize the OpenAI LLM with your API key
    llm = ChatOpenAI(
        model=MODEL,
        temperature=0.0,
        n=1
    )

    # Initialize memory and conversation
    memory = ConversationBufferWindowMemory()
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

def generate_code(conversation, prompt):
    print("Model response:")
    output = conversation.invoke(prompt)
    display_markdown(output['response'], raw=True)


## First Attempt at an XOR Approximator

We will construct a prompt that requests the LLM to generate a PyTorch neural network to approximate the [Exclusive Or](https://en.wikipedia.org/wiki/Exclusive_or). The truth table for the Exclusive Or (XOR) function is provided here:

```
0 XOR 0 = 0
1 XOR 0 = 1
0 XOR 1 = 1
1 XOR 1 = 0
```

If given data, neural networks can learn to approximate functions, so let's create a PyTorch neural network to approximate the XOR function.

In [3]:
conversation = start_conversation()
generate_code(conversation, """Write Python code to learn the XOR function with PyTorch.""")

Model response:


Certainly! To learn the XOR function using PyTorch, we'll need to set up a simple neural network. The XOR function is a classic problem in neural network training because it's not linearly separable, which means we'll need at least one hidden layer in our network.

Here's how you can set up a neural network in PyTorch to learn the XOR function:

```python
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Define the XOR class using PyTorch
class XORNet(nn.Module):
    def __init__(self):
        super(XORNet, self).__init__()
        # Neural network layers: 2 input neurons, 2 hidden neurons, 1 output neuron
        self.layer1 = nn.Linear(2, 2)
        self.layer2 = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.layer1(x))
        x = self.sigmoid(self.layer2(x))
        return x

# Initialize the model, loss function, and optimizer
model = XORNet()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Training data for XOR
data = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
labels = torch.tensor([[0.0], [1.0], [1.0], [0.0]])

# Train the model
for epoch in range(10000):
    # Forward pass: Compute predicted y by passing x to the model
    pred = model(data)

    # Compute and print loss
    loss = criterion(pred, labels)
    if epoch % 1000 == 0:
        print(f'Epoch {epoch} Loss: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Test the model
with torch.no_grad():
    test_pred = model(data)
    print("Predicted values:")
    print(test_pred)
```

### Notes on Execution:
- Ensure you have PyTorch installed in your Python environment. You can install it using `pip install torch`.
- The learning rate and the number of epochs are set to simple values for demonstration. You might need to adjust these depending on the convergence behavior.
- The model uses a sigmoid activation function, which is typical for binary classification tasks like XOR.
- The output after training should approximate the XOR truth table, with values close to 0 or 1.

# Requesting a Change to Generated Code

If you've taken my other course, you will know I prefer PyTorch sequences over extending the nn.Module class, at least for simple neural networks like an XOR approximator. LLMs do not share this opinion. However, the LLM will gladly humor me and generate a sequence. Here, I provide an additional prompt to request this rather than resubmitting a modified version of my first prompt.

In [4]:
generate_code(conversation, """Could you make use of a PyTorch sequence rather than a nn.Module class?""")

Model response:


Certainly! Using `torch.nn.Sequential` is a more concise way to define a model when you don't need the customizability that comes with defining your own `nn.Module` class. Here's how you can define the same XOR network using `Sequential`:

```python
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Define the XOR network using a sequential container
model = nn.Sequential(
    nn.Linear(2, 2),
    nn.Sigmoid(),
    nn.Linear(2, 1),
    nn.Sigmoid()
)

# Initialize the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Training data for XOR
data = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
labels = torch.tensor([[0.0], [1.0], [1.0], [0.0]])

# Train the model
for epoch in range(10000):
    # Forward pass: Compute predicted y by passing x to the model
    pred = model(data)

    # Compute and print loss
    loss = criterion(pred, labels)
    if epoch % 1000 == 0:
        print(f'Epoch {epoch} Loss: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Test the model
with torch.no_grad():
    test_pred = model(data)
    print("Predicted values:")
    print(test_pred)
```

### Notes on Execution:
- This version of the code uses `torch.nn.Sequential` to define the model in a more compact way. The functionality remains the same as the previous example.
- Ensure PyTorch is installed in your environment as mentioned previously.
- The learning rate and number of epochs are kept the same for consistency with the previous example. Adjust these parameters as needed based on the model's performance.
- The output should still approximate the XOR truth table, demonstrating the model's ability to learn the XOR function using a sequential model structure.

# Testing the Generated Code

LLMs are not overachievers; they will implement the code you ask for and not provide much more. When we run the XOR approximator's first version, the results are only sometimes accurate, especially if we run the program multiple times.

In [6]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Define the XOR network using a sequential container
model = nn.Sequential(
    nn.Linear(2, 2),
    nn.Sigmoid(),
    nn.Linear(2, 1),
    nn.Sigmoid()
)

# Initialize the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Training data for XOR
data = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
labels = torch.tensor([[0.0], [1.0], [1.0], [0.0]])

# Train the model
for epoch in range(10000):
    # Forward pass: Compute predicted y by passing x to the model
    pred = model(data)

    # Compute and print loss
    loss = criterion(pred, labels)
    if epoch % 1000 == 0:
        print(f'Epoch {epoch} Loss: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Test the model
with torch.no_grad():
    test_pred = model(data)
    print("Predicted values:")
    print(test_pred)

Epoch 0 Loss: 0.2688503861427307
Epoch 1000 Loss: 0.24990490078926086
Epoch 2000 Loss: 0.24987591803073883
Epoch 3000 Loss: 0.2498343586921692
Epoch 4000 Loss: 0.24977044761180878
Epoch 5000 Loss: 0.2496638000011444
Epoch 6000 Loss: 0.24946792423725128
Epoch 7000 Loss: 0.24906250834465027
Epoch 8000 Loss: 0.24809706211090088
Epoch 9000 Loss: 0.2454599142074585
Predicted values:
tensor([[0.4843],
        [0.5800],
        [0.4278],
        [0.4623]])


If you receive an error or the output is not exactly what you like, it is effective to provide that output and any errors to the LLM. Here, we provide the output and ask the LLM if that seems correct. Sometimes, the LLM may insist that the output is correct, so you must "debate" the LLM, providing additional details.

In [7]:
generate_code(conversation, """The output was:

Predicted values:
tensor([[0.4843],
        [0.5800],
        [0.4278],
        [0.4623]])

Are you sure that is correct?
""")

Model response:


It appears that the model hasn't learned the XOR function effectively, as the output values are not close to the expected binary outputs of 0 or 1. This could be due to several factors including insufficient training, the need for a different network architecture, or the need for tuning hyperparameters like the learning rate or the number of epochs.

Here are a few suggestions to improve the model's performance:

1. **Increase the number of epochs**: Sometimes, the network needs more training time to converge to the correct values.
2. **Adjust the learning rate**: If the learning rate is too high, the model might overshoot the minimum; if it's too low, it might not converge in a reasonable amount of time.
3. **Add more neurons in the hidden layer**: Increasing the capacity of the network can sometimes help in capturing the complexity of the function.
4. **Use a different optimizer**: Sometimes, changing the optimizer can lead to better results. For example, Adam is often more effective than SGD for many tasks.

Let's adjust the model by increasing the number of hidden neurons and using the Adam optimizer. Here's the revised code:

```python
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Define the XOR network using a sequential container
model = nn.Sequential(
    nn.Linear(2, 4),  # Increased the number of neurons in the hidden layer
    nn.Sigmoid(),
    nn.Linear(4, 1),
    nn.Sigmoid()
)

# Initialize the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)  # Changed to Adam optimizer

# Training data for XOR
data = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
labels = torch.tensor([[0.0], [1.0], [1.0], [0.0]])

# Train the model
for epoch in range(20000):  # Increased the number of epochs
    # Forward pass: Compute predicted y by passing x to the model
    pred = model(data)

    # Compute and print loss
    loss = criterion(pred, labels)
    if epoch % 1000 == 0:
        print(f'Epoch {epoch} Loss: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Test the model
with torch.no_grad():
    test_pred = model(data)
    print("Predicted values:")
    print(test_pred)
```

### Notes on Execution:
- The number of hidden neurons has been increased to 4.
- The optimizer has been changed from SGD to Adam.
- The number of training epochs has been doubled to 20,000.
- These changes are aimed at improving the model's ability to learn the XOR function more effectively. Adjust further if needed based on the output.

## Test the Improved Version

We now receive much more accurate output when we test the neural network provided.

In [8]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Define the XOR network using a sequential container
model = nn.Sequential(
    nn.Linear(2, 4),  # Increased the number of neurons in the hidden layer
    nn.Sigmoid(),
    nn.Linear(4, 1),
    nn.Sigmoid()
)

# Initialize the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)  # Changed to Adam optimizer

# Training data for XOR
data = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
labels = torch.tensor([[0.0], [1.0], [1.0], [0.0]])

# Train the model
for epoch in range(20000):  # Increased the number of epochs
    # Forward pass: Compute predicted y by passing x to the model
    pred = model(data)

    # Compute and print loss
    loss = criterion(pred, labels)
    if epoch % 1000 == 0:
        print(f'Epoch {epoch} Loss: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Test the model
with torch.no_grad():
    test_pred = model(data)
    print("Predicted values:")
    print(test_pred)

Epoch 0 Loss: 0.26264309883117676
Epoch 1000 Loss: 7.067574188113213e-05
Epoch 2000 Loss: 9.918729119817726e-06
Epoch 3000 Loss: 3.147976713080425e-06
Epoch 4000 Loss: 1.4494128208752954e-06
Epoch 5000 Loss: 7.723459702901891e-07
Epoch 6000 Loss: 4.419764536578441e-07
Epoch 7000 Loss: 2.627704134283704e-07
Epoch 8000 Loss: 1.595393825937208e-07
Epoch 9000 Loss: 9.79592442718058e-08
Epoch 10000 Loss: 6.045179645752796e-08
Epoch 11000 Loss: 3.737751441690307e-08
Epoch 12000 Loss: 2.3105236834908283e-08
Epoch 13000 Loss: 1.4265761016929446e-08
Epoch 14000 Loss: 8.795730366273347e-09
Epoch 15000 Loss: 5.41784128671452e-09
Epoch 16000 Loss: 3.3351594819208685e-09
Epoch 17000 Loss: 2.053748282548895e-09
Epoch 18000 Loss: 1.2684615580127456e-09
Epoch 19000 Loss: 7.867687057405703e-10
Predicted values:
tensor([[2.3532e-05],
        [9.9998e-01],
        [9.9998e-01],
        [2.2704e-05]])


## Combining the Conversation into a Single Prompt

We should combine this entire conversation into a single prompt, especially if we wish to save the prompt along with the code. We can request the LLM to create this combined prompt for us.

In [9]:
generate_code(conversation, """Okay, that is great, can you suggest a single
prompt that would have resulted in this last code output?""")

Model response:


Certainly! A single prompt that could lead directly to the last provided code output, which includes the modifications for improving the XOR function learning, might be:

"Can you provide a Python code using PyTorch to effectively learn the XOR function with increased hidden neurons, using the Adam optimizer, and with more training epochs?"

The LLM's attempt at a consoldated prompt is incomplete. It skips several important details and does not provide precise requirements. I will manually make some improvements, which you can see here.

In [11]:
# Start a new conversation
conversation = start_conversation()
generate_code(conversation, """
Can you provide Python code using PyTorch to effectively learn the XOR function
with 4 hidden neurons, using the Adam optimizer, and 20K training epochs?
Use a sequence not a nn.Module class.""")

Model response:


Certainly! Below is the Python code using PyTorch to learn the XOR function. This example uses a simple neural network with one hidden layer consisting of 4 neurons. The network is trained using the Adam optimizer over 20,000 epochs. The network architecture and training process are defined using a sequential model rather than subclassing `nn.Module`.

### Notes on the Code
- The XOR function is a simple binary classification problem where the output is true if the inputs are not equal.
- The network uses a single hidden layer with 4 neurons, which should be sufficient to learn the XOR pattern.
- ReLU activation is used for non-linearity in the hidden layer.
- The output layer uses a sigmoid activation function to output a probability between 0 and 1.
- Binary cross-entropy loss is used as it is suitable for binary classification tasks.
- The Adam optimizer is known for being effective in various conditions and is used here for optimization.
- The input and target tensors are explicitly defined to represent the XOR truth table.

```python
import torch
import torch.nn as nn
import torch.optim as optim

# Define the XOR inputs and outputs
inputs = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float)
targets = torch.tensor([[0], [1], [1], [0]], dtype=torch.float)

# Define the model using a sequential container
model = nn.Sequential(
    nn.Linear(2, 4),  # Input layer to hidden layer with 4 neurons
    nn.ReLU(),        # ReLU activation function
    nn.Linear(4, 1),  # Hidden layer to output layer
    nn.Sigmoid()      # Sigmoid activation function for binary output
)

# Define the loss function and the optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adam optimizer with learning rate of 0.01

# Training loop
for epoch in range(20000):  # 20,000 training epochs
    optimizer.zero_grad()   # Clear gradients for each training step
    outputs = model(inputs)  # Forward pass: compute predicted outputs by passing inputs to the model
    loss = criterion(outputs, targets)  # Compute loss
    loss.backward()  # Backward pass: compute gradient of the loss with respect to model parameters
    optimizer.step()  # Perform a single optimization step (parameter update)

    if (epoch + 1) % 1000 == 0:
        print(f'Epoch [{epoch + 1}/20000], Loss: {loss.item():.4f}')

# Testing the model
with torch.no_grad():  # Context-manager that disabled gradient calculation
    predicted = model(inputs).round()  # Forward pass and rounding off to get predictions
    print(f'Predicted tensor: {predicted}')
    print(f'Actual tensor: {targets}')
```

### Notes on Execution
- Ensure that PyTorch is installed in your Python environment to run this code.
- The learning rate and the number of epochs are set to typical starting values but may need adjustment based on the specific behavior of the training process.
- The print statements inside the loop provide a simple way to monitor the training progress.
- The final output shows the predicted values and actual values for the XOR function, allowing you to verify the model's performance.

## Test the Final Prompt

Now, we test the final prompt. My prompt produces an acceptable result, but there are some opportunities for improvement. You can specify the exact format for the output. For example, sometimes code is generated to round the results, but other times it is not.

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the XOR inputs and outputs
inputs = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float)
targets = torch.tensor([[0], [1], [1], [0]], dtype=torch.float)

# Define the model using a sequential container
model = nn.Sequential(
    nn.Linear(2, 4),  # Input layer to hidden layer with 4 neurons
    nn.ReLU(),        # ReLU activation function
    nn.Linear(4, 1),  # Hidden layer to output layer
    nn.Sigmoid()      # Sigmoid activation function for binary output
)

# Define the loss function and the optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adam optimizer with learning rate of 0.01

# Training loop
for epoch in range(20000):  # 20,000 training epochs
    optimizer.zero_grad()   # Clear gradients for each training step
    outputs = model(inputs)  # Forward pass: compute predicted outputs by passing inputs to the model
    loss = criterion(outputs, targets)  # Compute loss
    loss.backward()  # Backward pass: compute gradient of the loss with respect to model parameters
    optimizer.step()  # Perform a single optimization step (parameter update)

    if (epoch + 1) % 1000 == 0:
        print(f'Epoch [{epoch + 1}/20000], Loss: {loss.item():.4f}')

# Testing the model
with torch.no_grad():  # Context-manager that disabled gradient calculation
    predicted = model(inputs).round()  # Forward pass and rounding off to get predictions
    print(f'Predicted tensor: {predicted}')
    print(f'Actual tensor: {targets}')

Epoch [1000/20000], Loss: 0.0034
Epoch [2000/20000], Loss: 0.0009
Epoch [3000/20000], Loss: 0.0004
Epoch [4000/20000], Loss: 0.0002
Epoch [5000/20000], Loss: 0.0001
Epoch [6000/20000], Loss: 0.0001
Epoch [7000/20000], Loss: 0.0000
Epoch [8000/20000], Loss: 0.0000
Epoch [9000/20000], Loss: 0.0000
Epoch [10000/20000], Loss: 0.0000
Epoch [11000/20000], Loss: 0.0000
Epoch [12000/20000], Loss: 0.0000
Epoch [13000/20000], Loss: 0.0000
Epoch [14000/20000], Loss: 0.0000
Epoch [15000/20000], Loss: 0.0000
Epoch [16000/20000], Loss: 0.0000
Epoch [17000/20000], Loss: 0.0000
Epoch [18000/20000], Loss: 0.0000
Epoch [19000/20000], Loss: 0.0000
Epoch [20000/20000], Loss: 0.0000
Predicted tensor: tensor([[0.],
        [1.],
        [1.],
        [0.]])
Actual tensor: tensor([[0.],
        [1.],
        [1.],
        [0.]])
